In [ ]:
import sqlite3
import time
con = sqlite3.connect('database.sqlite')
cursorObj = con.cursor()

In [ ]:
#Question 1 Write a SQL query that lists all the players born between 1987 and 1990
#inclusive, sort them from the oldest to the youngest. The output of this query should be of the form:
#Player Name | Birthday

In [ ]:
t = time.process_time()
cursorObj.execute("""
SELECT player_name AS 'Player Name', 
       birthday    AS 'Birthday' 
FROM   player 
WHERE  birthday BETWEEN '1987-01-01' AND '1990-12-31' 
ORDER  BY birthday 
;""")
rows = cursorObj.fetchall() 
for row in rows: ## Let's show all the rows
  print(row)
t1_sql = time.process_time() - t

In [ ]:
#Question 2 (20 points) : Write a SQL query that ranks all countries and leagues based on the total
#amount of total goals scored per game in the whole dataset. Sort them by the largest to the smallest
#amount of goals. Note: Read this carefully. The output of this query should be of the form:
#Country | League Name |Total Goals Scored

In [ ]:
t = time.process_time()
cursorObj.execute("""
SELECT c.NAME, 
       l.NAME, 
       Sum(home_team_goal + away_team_goal) AS Goals 
FROM   match m, 
       league l, 
       country c 
WHERE  m.league_id = l.id 
       AND m.country_id = c.id 
GROUP  BY c.NAME, 
          l.NAME 
ORDER  BY goals DESC 
;""")
rows = cursorObj.fetchall() 
for row in rows: ## Let's show all the rows
  print(row)
t2_sql = time.process_time() - t

In [ ]:
#Question 3 Write a SQL query that ranks all teams by the average of all their attributes (not
#the players' attributes), sort them from best to worst. The output of this query should be of the form:
#Team Long Name | Average of Attributes

In [ ]:
t = time.process_time()
cursorObj.execute("""
SELECT c.team_long_name, 
       Avg(some) AS 'Average_of_Attributes' 
FROM   (SELECT a.team_long_name, 
               Ifnull(b.buildupplayspeed, '0') 
               + Ifnull(b.buildupplaydribbling, '0') 
               + Ifnull(b.buildupplaypassing, '0') 
               + Ifnull(b.chancecreationpassing, '0') 
               + Ifnull(b.chancecreationcrossing, '0') 
               + Ifnull(b.chancecreationshooting, '0') 
               + Ifnull(b.defencepressure, '0') 
               + Ifnull(b.defenceaggression, '0') 
               + Ifnull(b.defenceteamwidth, '0') AS 'some' 
        FROM   team AS a, 
               team_attributes AS b 
        WHERE  a.team_fifa_api_id = b.team_fifa_api_id) AS c 
GROUP  BY c.team_long_name 
ORDER  BY average_of_attributes DESC
;""")
rows = cursorObj.fetchall() 
for row in rows: ## Let's show all the rows
  print(row)
t3_sql = time.process_time() - t

In [ ]:
#Question 4 (20 points) : Write a SQL query that ranks all teams by the average of their players' attributes, sort them by 
#descending order displaying only the top 5. The output of this query should be of the form:
#Team Name| Number of Players | Player Attribute Average

In [ ]:
#Player attributes make team attributes
t = time.process_time()
cursorObj.execute("""
SELECT  overall_rating + potential + crossing + finishing + 
			heading_accuracy + short_passing, + volleys +
			dribbling + curve + free_kick_accuracy + long_passing +
			ball_control + acceleration + sprint_speed + agility +
			reactions + balance + shot_power + jumping + stamina +
			strength + long_shots + aggression + interceptions +
			positioning + vision + penalties + marking +
			standing_tackle + sliding_tackle + gk_diving + gk_reflexes + 
			gk_handling + gk_kicking + gk_positioning as Attribute
FROM Player_Attributes
ORDER BY Attribute DESC
LIMIT 5
;""")
rows = cursorObj.fetchall() 
for row in rows: ## Let's show all the rows
  print(row)
t4_sql = time.process_time() - t

In [ ]:
#Question 5 (40 points) : Write a SINGLE SQL query that finds the date that had the most goals scored on,
#per each different season and league. The output of this query should be of the form:
#Date (dd/mm/yy) | Season | League Name | Goals scored

In [ ]:
t = time.process_time()
cursorObj.execute("""
SELECT date, 
       season, 
       ln, 
       Max(goals) 
FROM   (SELECT Strftime('%d/%m/%Y', m.date)         AS Date, 
               m.season                             AS 'Season', 
               l.NAME                               AS 'LN', 
               Sum(home_team_goal + away_team_goal) AS Goals 
        FROM   match m, 
               league l 
        WHERE  m.league_id = l.id 
        GROUP  BY m.date, 
                  l.NAME, 
                  m.season 
        ORDER  BY goals DESC) 
GROUP  BY ln, 
          season 
ORDER  BY goals DESC 
;""")
rows = cursorObj.fetchall() 
for row in rows: ## Let's show all the rows
  print(row)
t5_sql = time.process_time() - t

In [ ]:
#Question:6 Graduate Student Task (40 points) : Write a SINGLE SQL query that finds the top 5 teams in terms of
#goals scored PER league for the 2008/2009 season. The output of this query should be of the form:
#Season | League | Rank | Team Name | Goals Scored

In [ ]:
t = time.process_time()
cursorObj.execute("""
SELECT * 
FROM   (SELECT AL.season, 
               AL.league, 
               Rank() 
                 OVER ( 
                   partition BY AL.league 
                   ORDER BY totalgoals DESC) 'TeamRank', 
               AL.team, 
               AL.totalgoals 
        FROM   (SELECT Scores.season, 
                       Scores.team, 
                       Scores.league, 
                       Sum(Scores.goals) AS TotalGoals 
                FROM   (SELECT m.season            AS Season, 
                               t.team_long_name    AS 'Team', 
                               l.NAME              AS 'League', 
                               Sum(home_team_goal) AS Goals, 
                               'Home'              AS 'Location' 
                        FROM   match m, 
                               team t, 
                               league l 
                        WHERE  m.home_team_api_id = t.team_api_id 
                               AND m.country_id = l.country_id 
                               AND m.season = '2008/2009' 
                        GROUP  BY l.NAME, 
                                  t.team_long_name 
                        UNION 
                        SELECT m.season            AS Season, 
                               t.team_long_name    AS 'Team', 
                               l.NAME              AS 'League', 
                               Sum(away_team_goal) AS Goals, 
                               'Away'              AS 'Location' 
                        FROM   match m, 
                               team t, 
                               league l 
                        WHERE  m.away_team_api_id = t.team_api_id 
                               AND m.country_id = l.country_id 
                               AND m.season = '2008/2009' 
                        GROUP  BY l.NAME, 
                                  t.team_long_name) AS Scores 
                GROUP  BY Scores.league, 
                          Scores.team) AS AL) AS TR 
WHERE  TR.teamrank < 6 
ORDER  BY TR.teamrank    
;""")
rows = cursorObj.fetchall() 
for row in rows: ## Let's show all the rows
  print(row)
t6_sql = time.process_time() - t

In [ ]:
#Execution time of SQL queries #seconds
print(t1_sql)
print(t2_sql)
print(t3_sql)
print(t4_sql)
print(t5_sql)
print(t6_sql)

In [ ]:
#Pyspark Session
from pyspark.sql import SparkSession
import re
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
import pandas as pd
conn = sqlite3.connect("database.sqlite")
df_team = pd.read_sql_query("SELECT* FROM Team", conn)
df_player = pd.read_sql_query("SELECT* FROM Player", conn)
df_team_a = pd.read_sql_query("SELECT* FROM Team_Attributes", conn)
df_player_a = pd.read_sql_query("SELECT* FROM Player_Attributes", conn)
df_Country = pd.read_sql_query("SELECT* FROM Country", conn)
df_League = pd.read_sql_query("SELECT* FROM League", conn)
df_Match = pd.read_sql_query("SELECT* FROM Match", conn)